In [12]:
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import pydicom
import skimage.io
import skimage.transform
import skimage.color
import skimage
import cv2
from PIL import Image

import model
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset,  collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader
from mydataloader import MyDataset
import coco_eval
import csv_eval
import visdom
vis = visdom.Visdom(env='retinanet_1fold')

assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#参数
NAME="RSNA"
DATA_PATH = "/data/krf/dataset"
CSV_TRAIN = DATA_PATH + "/csv_train.csv"
CSV_VAL = DATA_PATH + "/csv_val.csv"
CSV_CLASSES = DATA_PATH + "/classes.csv"
TEST_PATH = DATA_PATH +"/stage_1_test_images"
DEPTH = 50
EPOCHS = 10

VAL_SIZE = 100
TRAIN_SIZE = 500

CUDA available: True


In [13]:
#数据预处理
import csv
import random
with open(DATA_PATH+"/stage_1_train_labels.csv") as f:
    reader = csv.reader(f)
    rows=[row for row in  reader]
    rows = rows[1:TRAIN_SIZE]
    random.shuffle(rows)
    for row in rows:
        row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
        if row[1] == '' and row[2] == '' and row[3] == '' and row[4] == '':
            row[5] = ''
        else:
            row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
            row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
    val_rows = rows[:VAL_SIZE]
    train_rows = rows[VAL_SIZE:]
    print(len(val_rows),len(train_rows))
    with open(CSV_TRAIN,'w') as f2:
        write = csv.writer(f2)
        write.writerows(train_rows)
        print("csv_train 写入完毕")
    with open(CSV_VAL,'w') as f3:
        write = csv.writer(f3)
        write.writerows(val_rows)
        print("csv_val 写入完毕")

with open(CSV_CLASSES,'w') as f:
    write = csv.writer(f)
    row = ['1','0']
    write.writerow(row)
    print("csv_classes 写入完毕")

100 399
csv_train 写入完毕
csv_val 写入完毕
csv_classes 写入完毕


In [14]:
#%time
#制作数据loader
dataset_train = MyDataset(train_file=CSV_TRAIN, class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
dataset_val = MyDataset(train_file=CSV_VAL, class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Resizer()]))

#每次的sampler的参数：来源、batchsize、是否抛弃最后一层？？？
sampler = AspectRatioBasedSampler(dataset_train, batch_size=2, drop_last=False)
# num_workers 同时工作的组？collater:校验用的吧
dataloader_train = DataLoader(dataset_train, num_workers=1, collate_fn=collater, batch_sampler=sampler)

# if dataset_val is not None:
#     sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
#     dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)

dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)




In [15]:
# Create the model
if DEPTH == 18:
    retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
elif DEPTH == 34:
    retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
elif DEPTH == 50:
    retinanet = model.resnet50(num_classes=dataset_train.num_classes(), pretrained=True)
elif DEPTH == 101:
    retinanet = model.resnet101(num_classes=dataset_train.num_classes(), pretrained=True)
elif DEPTH == 152:
    retinanet = model.resnet152(num_classes=dataset_train.num_classes(), pretrained=True)
else:
    raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()
#变成并行
retinanet = torch.nn.DataParallel(retinanet).cuda()
#训练模式
retinanet.training = True
#学习率0.00001  
optimizer = optim.Adam(retinanet.parameters(), lr=1e-5)
#如果3个epoch损失没有减少则降低学习率
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
# TODO 这是干什么 deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈：这里是定义了一个500的队列
loss_hist = collections.deque(maxlen=500)

In [17]:
retinanet.train()
#BN层冻结！！
retinanet.module.freeze_bn()

print('Num training images: {}'.format(len(dataset_train)))
#vis.line(X=x, Y=y, win='polynomial', update='append' if ii > 0 else None ,opts={'title':'append'})
count = 0
val_freq = 10
for epoch_num in range(EPOCHS):

    retinanet.train()
    retinanet.module.freeze_bn()
    
    epoch_loss = []
    
    for iter_num, data in enumerate(dataloader_train):
#         try:
        optimizer.zero_grad()

        classification_loss, regression_loss = retinanet([Variable(data['img'].cuda().float()), Variable(data['annot'].cuda())])
        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()

        loss = classification_loss + regression_loss

        if bool(loss == 0):
            continue
        #反向传播？
        loss.backward()

        #这是干嘛？？TODO
        torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)

        #这?TODO
        optimizer.step()

        loss_hist.append(float(loss))

        epoch_loss.append(float(loss))

        print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))
        vis.line(X=torch.Tensor([count]), Y=torch.Tensor([np.mean(loss_hist)]), win='train loss', update='append' ,opts={'title':'train loss'})
        vis.save(['retinanet_1fold'])
        del classification_loss
        del regression_loss
        count+=1
        if count % val_freq == 0:
            retinanet.eval()
            print("Evaluating dataset")
            mAP = csv_eval.evaluate(dataset_val,retinanet)
            vis.line(X=torch.Tensor([count]), Y=torch.Tensor([mAP[0][0]]), win='val mAP', update='append' ,opts={'title':'mAP val'})
            vis.save(['retinanet_1fold'])
            retinanet.train()
#         except Exception as e:
#             print(e)
#             continue
        

    
#     retinanet.eval()
#     for index,data in enumerate(dataloader_val):
#         #data = dataset[index]
#         scale = data['scale']
#         # run network
#         #scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
#         print(Variable(data['img'].cuda()))
#         scores, labels, boxes = retinanet(Variable(data['img'].cuda()))
#         #mAP = csv_eval.evaluate(dataset_val,retinanet)
#     try:
        
# #         print("mAP=",mAP)
#     except Exception as e:
#         print("Exception:",e)
#         continue
        
    #这一步也看不懂？？TODO 
    scheduler.step(np.mean(epoch_loss))
    
    #torch.save(retinanet.module, 'weights/{}_retinanet_{}.pt'.format(NAME, epoch_num))
#     torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(parser.dataset, epoch_num))

retinanet.eval()

#torch.save(retinanet, 'weights/model_final.pt'.format(epoch_num))

Num training images: 347
Epoch: 0 | Iteration: 3 | Classification loss: 0.42046 | Regression loss: 0.57989 | Running loss: 1.09883
Epoch: 0 | Iteration: 4 | Classification loss: 0.56363 | Regression loss: 0.46890 | Running loss: 1.09676
Epoch: 0 | Iteration: 6 | Classification loss: 0.31376 | Regression loss: 0.70795 | Running loss: 1.09449
Epoch: 0 | Iteration: 10 | Classification loss: 0.49545 | Regression loss: 0.52017 | Running loss: 1.09217
Epoch: 0 | Iteration: 13 | Classification loss: 0.33626 | Regression loss: 0.52954 | Running loss: 1.08570
Epoch: 0 | Iteration: 14 | Classification loss: 0.36825 | Regression loss: 0.44376 | Running loss: 1.07810
Epoch: 0 | Iteration: 16 | Classification loss: 0.81573 | Regression loss: 1.02671 | Running loss: 1.09876
Epoch: 0 | Iteration: 17 | Classification loss: 0.32538 | Regression loss: 0.41896 | Running loss: 1.08943
Epoch: 0 | Iteration: 20 | Classification loss: 0.24698 | Regression loss: 0.45246 | Running loss: 1.07943
Epoch: 0 | Iter

Epoch: 0 | Iteration: 118 | Classification loss: 0.81013 | Regression loss: 1.09477 | Running loss: 0.95744
Epoch: 0 | Iteration: 120 | Classification loss: 0.21399 | Regression loss: 0.45738 | Running loss: 0.95471
Epoch: 0 | Iteration: 121 | Classification loss: 0.17553 | Regression loss: 0.53018 | Running loss: 0.95236
Epoch: 0 | Iteration: 122 | Classification loss: 0.60116 | Regression loss: 1.00684 | Running loss: 0.95849
Epoch: 0 | Iteration: 123 | Classification loss: 0.54720 | Regression loss: 1.03701 | Running loss: 0.96428
Epoch: 0 | Iteration: 129 | Classification loss: 0.17638 | Regression loss: 0.44787 | Running loss: 0.96117
Epoch: 0 | Iteration: 130 | Classification loss: 0.45264 | Regression loss: 0.54182 | Running loss: 0.96147
Evaluating dataset
93/93
mAP:
1: 0.03215943917664189
Epoch: 0 | Iteration: 131 | Classification loss: 0.23459 | Regression loss: 0.50676 | Running loss: 0.95948
Epoch: 0 | Iteration: 133 | Classification loss: 0.20781 | Regression loss: 0.47247

Epoch: 1 | Iteration: 72 | Classification loss: 0.13384 | Regression loss: 0.41097 | Running loss: 0.92207
Epoch: 1 | Iteration: 75 | Classification loss: 0.14368 | Regression loss: 0.43821 | Running loss: 0.92016
Epoch: 1 | Iteration: 76 | Classification loss: 0.40830 | Regression loss: 0.47300 | Running loss: 0.91994
Epoch: 1 | Iteration: 77 | Classification loss: 0.33329 | Regression loss: 0.49757 | Running loss: 0.91945
Evaluating dataset
93/93
mAP:
1: 0.06080431826602069
Epoch: 1 | Iteration: 78 | Classification loss: 0.38150 | Regression loss: 0.43637 | Running loss: 0.91889
Epoch: 1 | Iteration: 82 | Classification loss: 0.21315 | Regression loss: 0.41328 | Running loss: 0.91728
Epoch: 1 | Iteration: 83 | Classification loss: 0.26357 | Regression loss: 0.46412 | Running loss: 0.91624
Epoch: 1 | Iteration: 85 | Classification loss: 0.20089 | Regression loss: 0.37943 | Running loss: 0.91442
Epoch: 1 | Iteration: 86 | Classification loss: 0.11297 | Regression loss: 0.42089 | Runnin

Epoch: 2 | Iteration: 17 | Classification loss: 0.10429 | Regression loss: 0.31780 | Running loss: 0.86214
Evaluating dataset
93/93
mAP:
1: 0.05698008743990824
Epoch: 2 | Iteration: 18 | Classification loss: 0.23228 | Regression loss: 0.36197 | Running loss: 0.86108
Epoch: 2 | Iteration: 19 | Classification loss: 0.17060 | Regression loss: 0.42946 | Running loss: 0.86004
Epoch: 2 | Iteration: 21 | Classification loss: 0.15513 | Regression loss: 0.43818 | Running loss: 0.85899
Epoch: 2 | Iteration: 22 | Classification loss: 0.15763 | Regression loss: 0.34904 | Running loss: 0.85760
Epoch: 2 | Iteration: 24 | Classification loss: 0.23008 | Regression loss: 0.46072 | Running loss: 0.85694
Epoch: 2 | Iteration: 26 | Classification loss: 0.09837 | Regression loss: 0.24580 | Running loss: 0.85494
Epoch: 2 | Iteration: 29 | Classification loss: 0.16526 | Regression loss: 0.29616 | Running loss: 0.85341
Epoch: 2 | Iteration: 30 | Classification loss: 0.17775 | Regression loss: 0.48449 | Runnin

Epoch: 2 | Iteration: 140 | Classification loss: 0.20949 | Regression loss: 0.42871 | Running loss: 0.81147
Epoch: 2 | Iteration: 142 | Classification loss: 0.12087 | Regression loss: 0.37878 | Running loss: 0.81051
Epoch: 2 | Iteration: 144 | Classification loss: 0.09960 | Regression loss: 0.38452 | Running loss: 0.80950
Epoch: 2 | Iteration: 145 | Classification loss: 0.20311 | Regression loss: 0.72563 | Running loss: 0.80987
Epoch: 2 | Iteration: 146 | Classification loss: 0.14877 | Regression loss: 0.32348 | Running loss: 0.80884
Epoch: 2 | Iteration: 150 | Classification loss: 0.12257 | Regression loss: 0.30140 | Running loss: 0.80766
Epoch: 2 | Iteration: 152 | Classification loss: 0.11746 | Regression loss: 0.28351 | Running loss: 0.80643
Epoch: 2 | Iteration: 156 | Classification loss: 0.14493 | Regression loss: 0.32930 | Running loss: 0.80542
Evaluating dataset


Process Process-15:
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/queues.py", line 104, in get
    if timeout < 0 or not self._poll(timeout):
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/connection.py", line 911

KeyboardInterrupt: 

AttributeError: 'str' object has no attribute 'copy'

In [6]:
#######################################################KRF CREATED 2018/11/15###################################################
class TestDataset(Dataset):
    """Test dataset.类似于MyDataset，不过没有csv和label"""

    def __init__(self,test_fp, transform=None):
        """
        Args:
            test_fp (string):训练集的文件目录
            
        """
        self.test_fp = test_fp
        self.transform = transform
        self.image_names = os.listdir(self.test_fp)[:100] #只测试100张
        for i in range(len(self.image_names)):
            self.image_names[i] =  os.path.join(TEST_PATH,self.image_names[i])

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):

        img = self.load_image(idx)
        sample = {'img': img,'name' : self.image_names[idx]}
        if self.transform:
            sample = self.transform(sample)
        return sample

    def load_image(self, image_index):
#         img = skimage.io.imread(self.image_names[image_index])
        ds = pydicom.read_file(self.image_names[image_index])
        img = ds.pixel_array
        
        if len(img.shape) == 2:
            img = skimage.color.gray2rgb(img)

        return img.astype(np.float32)/255.0
    
    #####################################Modified by KRF######################
    def image_aspect_ratio(self, image_index):
        #image = Image.open(self.image_names[image_index])
        ds = pydicom.read_file(self.image_names[image_index])
        img_arr = ds.pixel_array
        image = Image.fromarray(img_arr).convert('RGB')
        return float(image.width) / float(image.height)
###################################################################################################################################

In [7]:
#覆盖另一个
class NormalizerTest(object):

    def __init__(self):
        self.mean = np.array([[[0.485, 0.456, 0.406]]])
        self.std = np.array([[[0.229, 0.224, 0.225]]])

    def __call__(self, sample):

        image, name = sample['img'], sample['name']

        return {'img':((image.astype(np.float32)-self.mean)/self.std), 'name': name}
class ResizerTest(object):
    """Convert ndarrays in sample to Tensors."""

    #def __call__(self, sample, min_side=608, max_side=1024):
    ###########################################KRF Modeified###########################################################
    def __call__(self, sample, min_side=512, max_side=1024):
        image, name = sample['img'], sample['name']

        rows, cols, cns = image.shape

        smallest_side = min(rows, cols)

        # rescale the image so the smallest side is min_side
        scale = min_side / smallest_side

        # check if the largest side is now greater than max_side, which can happen
        # when images have a large aspect ratio
        largest_side = max(rows, cols)

        if largest_side * scale > max_side:
            scale = max_side / largest_side

        # resize the image with the computed scale
        image = skimage.transform.resize(image, (int(round(rows*scale)), int(round((cols*scale)))))
        rows, cols, cns = image.shape

        pad_w = 32 - rows%32
        pad_h = 32 - cols%32

        new_image = np.zeros((rows + pad_w, cols + pad_h, cns)).astype(np.float32)
        new_image[:rows, :cols, :] = image.astype(np.float32)

        #annots[:, :4] *= scale

        return {'img': torch.from_numpy(new_image), 'name': name, 'scale': scale}

dataset_test = TestDataset(TEST_PATH,transform=transforms.Compose([NormalizerTest(), ResizerTest()]))
print(len(dataset_test))

100


In [15]:
# def detect(dataset, retinanet, score_threshold=0.05, max_detections=100, save_path=None):
#     """ Get the detections from the retinanet using the generator.
#     The result is a list of lists such that the size is:
#         all_detections[num_images][num_classes] = detections[num_detections, 4 + num_classes]
#     # Arguments
#         dataset         : The generator used to run images through the retinanet.
#         retinanet           : The retinanet to run on the images.
#         score_threshold : The score confidence threshold to use.
#         max_detections  : The maximum number of detections to use per image.
#         save_path       : The path to save the images with visualized detections to.
#     # Returns
#         A list of lists containing the detections for each image in the generator.
#     """
#     all_detections = [[None for i in range(dataset.num_classes())] for j in range(len(dataset))]
    
#     retinanet.eval()
    
#     with torch.no_grad():

#         for index in range(len(dataset)):
        
#             data = dataset[index]
#             scale = data['scale']

#             # run network
#             scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
#             #scores, labels, boxes = retinanet(data['img'].cuda().float())
#             scores = scores.cpu().numpy()
#             labels = labels.cpu().numpy()
#             boxes  = boxes.cpu().numpy()

#             # correct boxes for image scale
#             boxes /= scale

#             # select indices which have a score above the threshold
#             indices = np.where(scores > score_threshold)[0]
#             if indices.shape[0] > 0:
#                 # select those scores
#                 scores = scores[indices]

#                 # find the order with which to sort the scores
#                 scores_sort = np.argsort(-scores)[:max_detections]

#                 # select detections
#                 image_boxes      = boxes[indices[scores_sort], :]
#                 image_scores     = scores[scores_sort]
#                 image_labels     = labels[indices[scores_sort]]
#                 image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

#                 # copy detections to all_detections
#                 for label in range(dataset.num_classes()):
#                     all_detections[index][label] = image_detections[image_detections[:, -1] == label, :-1]
#             else:
#                 # copy detections to all_detections
#                 for label in range(dataset.num_classes()):
#                     all_detections[index][label] = np.zeros((0, 5))

#             print('{}/{}'.format(index + 1, len(dataset)), end='\r')

#     return all_detections

In [8]:
#retinanet.load_state_dict('weights/RSNA_retinanet_3.pt')
def collaterTest(data):

    imgs = [s['img'] for s in data]
    names = [s['name'] for s in data]
    scales = [s['scale'] for s in data]
        
    widths = [int(s.shape[0]) for s in imgs]
    heights = [int(s.shape[1]) for s in imgs]
    batch_size = len(imgs)

    max_width = np.array(widths).max()
    max_height = np.array(heights).max()

    padded_imgs = torch.zeros(batch_size, max_width, max_height, 3)

    for i in range(batch_size):
        img = imgs[i]
        padded_imgs[i, :int(img.shape[0]), :int(img.shape[1]), :] = img

#    max_num_annots = max(annot.shape[0] for annot in annots)
    
#     if max_num_annots > 0:

#         annot_padded = torch.ones((len(annots), max_num_annots, 5)) * -1

#         if max_num_annots > 0:
#             for idx, annot in enumerate(annots):
#                 #print(annot.shape)
#                 if annot.shape[0] > 0:
#                     annot_padded[idx, :annot.shape[0], :] = annot
#     else:
#         annot_padded = torch.ones((len(annots), 1, 5)) * -1


    padded_imgs = padded_imgs.permute(0, 3, 1, 2)

    return {'img': padded_imgs, 'names': names, 'scale': scales}


sampler_test = AspectRatioBasedSampler(dataset_test, batch_size=1, drop_last=False)
dataloader_test = DataLoader(dataset_test, num_workers=1, collate_fn=collaterTest, batch_sampler=sampler_test)

retinanet = torch.load("weights/RSNA_retinanet_3.pt")

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()

retinanet.eval()

unnormalize = UnNormalizer()

# def draw_caption(image, box, caption):

#     b = np.array(box).astype(int)
#     cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
#     cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)



In [15]:
filepath = "submission.csv"
with open(filepath, 'w') as file:
    file.write("patientId,PredictionString\n")
    for idx, data in enumerate(dataloader_test):
        patientId = os.path.splitext(os.path.basename(data['names'][0]))[0]
        print(patientId)
        file.write(patientId+",")
        with torch.no_grad():
            st = time.time()
            scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
            print('Elapsed time: {}'.format(time.time()-st))
            idxs = np.where(scores>0.5)
            img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

            img[img<0] = 0
            img[img>255] = 255

            img = np.transpose(img, (1, 2, 0))

#             img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)

            outstr = ""#输出到csv中
            resize_factor = 2 #输入的是512，原图像是1024
            for j in range(idxs[0].shape[0]):
                bbox = transformed_anchors[idxs[0][j], :]
                x1 = int(bbox[0])
                y1 = int(bbox[1])
                x2 = int(bbox[2])
                y2 = int(bbox[3])
                #label_name = dataset_val.labels[int(classification[idxs[0][j]])]
#                 draw_caption(img, (x1, y1, x2, y2), "1")

#                 cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)
                #print(label_name)
                width = x2 - x1;
                height = y2 - y1
                outstr += ' '
                s = scores[idxs[0][j]].cpu().numpy()
                print(s)
                outstr += str(s)
                outstr += ' '
                bboxes_str = "{} {} {} {}".format(x1*resize_factor, y1*resize_factor, \
                                                               width*resize_factor, height*resize_factor)
                outstr += bboxes_str
#             cv2.imshow('img', img)
#             cv2.waitKey(0)
            file.write(outstr+"\n")
        

2853a995-743e-43d8-9ea4-068ffc6bf420
Elapsed time: 0.16528034210205078
29f74eff-c4f5-49d7-9ef8-fdb051100539
Elapsed time: 0.06557154655456543
21e696f2-9d4e-42b0-8fe6-e4e59e667476
Elapsed time: 0.05134177207946777
02ae356b-c39e-474f-908f-0af600e6b03d
Elapsed time: 0.05063629150390625
0.5670697
0.5359563
11fc3649-c448-488a-8bf2-412ef14971d4
Elapsed time: 0.07328963279724121
0050f8bb-36a4-4a1a-8de5-2d73154c2571
Elapsed time: 0.1954667568206787
0e85807d-4f11-43eb-9bf8-deecfcf79961
Elapsed time: 0.14347481727600098
20f0b519-4336-4f74-b03d-ddc95d454bdb
Elapsed time: 0.07823443412780762
2e5ab719-7447-4d3f-95f8-5ed5c8c736a1
Elapsed time: 0.055680274963378906
135a0985-d0f1-4c94-af72-f11e18c30bd5
Elapsed time: 0.07593512535095215
0.58536214
0.51516116
268833a7-38e7-4c97-a4f1-db5175ea34ae
Elapsed time: 0.08113455772399902
237ce1d8-6ff1-4cf7-afa1-be822a58276c
Elapsed time: 0.06859779357910156
2bd3afa7-414c-431f-ad98-0d1b03f175b8
Elapsed time: 0.07580804824829102
1328d56a-5b37-41b2-bc1b-a6c90989c64